This the benchmarking against blast. We test the accuracy of blast in predicting the transporter (to compare to our classification model) by running a sequence against the database of the training dataset and seeing if the highest similarity sequence is a transporter or not. If the highest similarity sequence in the training dataset is a transport, we classify the input sequence as a transporter 

We test the function of the function that takes an input sequence and compares it all sequences in training.

In [1]:
import pandas as pd
import subprocess

In [2]:
from Bio import Blast
from Bio import SeqIO
from Bio.Blast.Applications import NcbimakeblastdbCommandline, NcbiblastnCommandline, NcbiblastpCommandline 
from Bio.Blast import NCBIXML

/u/sshafaei1/.conda/envs/newmain/lib/python3.10/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


load in the training dataset as reference


In [ ]:
ref_data = pd.read_csv("./dataset/final_training.csv")
print(len(ref_data))
ref_data.head(n=1)

117766


,Unnamed: 0,Sequence,output,ankh_large
0,0,MENLDEVQWRSPEWIQAHQGLRTDNVLDYFAESPFYDRVSNNQVLR...,0,"[0.018960073590278625, 0.0004224804579280317, ..."


In [8]:
ref_data["id"] = list(range(0,len(ref_data)))
ref_data.head(n=1)

,Unnamed: 0,Sequence,output,ankh_large,id
0,0,MENLDEVQWRSPEWIQAHQGLRTDNVLDYFAESPFYDRVSNNQVLR...,0,"[0.018960073590278625, 0.0004224804579280317, ...",0


creating the database, first create a fasta file, then merge it

In [9]:
f= open("./training.fasta","w")
for i, row in ref_data.iterrows():
    #get current sequence entry code, with the classification output
    true_class = row["output"]#real classification
    entry = row["id"]
    seq = row["Sequence"]
    f.write(f">{entry}-{str(true_class)}\n")#write the heading name includes true classification as a -
    f.write(f"{seq}\n") #write the sequence
f.close()

Creating the blast db file


In [10]:
subprocess.run([
    "makeblastdb",
    "-in", "training.fasta",
    "-dbtype", "prot",
    "-out", "my_protein_db"
], check=True)



Building a new DB, current time: 04/11/2025 00:43:37
New DB name:   /work/hdd/bcij/sshafaei1/Transporter_Protein_project/Transporter_classification_model_3/Blast_benchmarking/my_protein_db
New DB title:  training.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /work/hdd/bcij/sshafaei1/Transporter_Protein_project/Transporter_classification_model_3/Blast_benchmarking/my_protein_db
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 117766 sequences in 14.2132 seconds.




CompletedProcess(args=['makeblastdb', '-in', 'training.fasta', '-dbtype', 'prot', '-out', 'my_protein_db'], returncode=0)

### Testing running test data

running on test data(mostly run on cluster, testing here)

In [ ]:
input_data = pd.read_csv("./dataset/final_testing.csv")
input_data["id"] = list(range(0,len(input_data))) #add id column

print(len(input_data))

26282


In [14]:

f= open("./outputs.csv","w") #create a file file to store outputs
f.write("seq,true,predicted\n") #write header
for i, row in input_data.iterrows():
    if i > 6: break
    #get current sequence entry code, with the classification output
    true_class = row["output"]#real classification
    entry = row["id"]
    seq = row["Sequence"]
    #create a single query fasta
    q = open("./query.fasta","w")
    q.write(">test\n")
    q.write(f"{seq}\n") 
    q.close()

    #run a blast search
    blastp_cline = NcbiblastpCommandline(
        query="query.fasta", 
        db="my_protein_db", 
        evalue=0.001, 
        outfmt=5,  # XML format
        out="results.xml"
    )
    stdout, stderr = blastp_cline()
    if stderr:
        subprocess.run(["echo", f"error: {stderr}"])
    #Parse BLAST Results
    with open("results.xml") as result_handle:
        blast_record = NCBIXML.read(result_handle)

    # Step 5: Get Most Similar Sequence
    predicted_class = 0
    try:
        best_match = blast_record.alignments[0]  # Top hit
        predicted_class = str(best_match.title).split("-")[1]
        subprocess.run(["echo", f"predicted: {predicted_class}"])
    except:
        subprocess.run(["echo", "No significant matches found."])
        continue

    f.write(f"{seq},{true_class},{predicted_class}\n")#write rows to the csv
f.close()

predicted: 0
No significant matches found.
predicted: 0
No significant matches found.
No significant matches found.
predicted: 1
predicted: 0
